# Copernicus Sentinel-2 Data Search and Exploration

This Jupyter Notebook demonstrates how to search, filter, and explore Sentinel-2 satellite imagery metadata using the `phidown` library. The workflow includes:

- **Setting Search Parameters:**  
    The notebook defines parameters such as collection name (`SENTINEL-2`), product type (`S2MSI1C`), cloud cover threshold, area of interest (AOI) in WKT format, date range, and sorting options.

- **Executing the Search:**  
    An instance of `CopernicusDataSearcher` is configured with the specified parameters and used to query the Copernicus Open Access Hub for matching Sentinel-2 products.

- **Viewing Results:**  
    The resulting metadata is loaded into a pandas DataFrame (`df`). Key columns such as `Id`, `Name`, `S3Path`, and `GeoFootprint` are displayed for quick inspection.

## Variables and Data

- **aoi_wkt:** AOI polygon in WKT format. Polygon must start and end with the same point. Coordinates must be given in EPSG 4326
- **cloud_cover_threshold:** Maximum allowed cloud cover percentage.
- **collection_name:** Data collection to search (`SENTINEL-2`).
- **columns_to_display:** List of DataFrame columns to display.
- **df:** DataFrame containing search results (1000 entries, 16 columns).
- **searcher:** Configured `CopernicusDataSearcher` instance.
- **start_date, end_date:** Date range for the search.
- **product_type, orbit_direction, order_by, top:** Additional search parameters.

In [ ]:
%load_ext autoreload
%autoreload 2

# Search

You can modify the search parameters to refine your query, re-run the search, and further analyze or visualize the results as needed.

### Sentinel Products

In [ ]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

aoi_wkt = "POLYGON((-95.0 29.0, -95.0 30.0, -94.0 30.0, -94.0 29.0, -95.0 29.0))"  # Define the AOI in WKT format

searcher = CopernicusDataSearcher()
# Configure the search parameters
searcher._query_by_filter(
    collection_name='SENTINEL-1',
    product_type='SLC',
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=None,  # Example: aoi_wkt=aoi_wkt if you want to use the defined AOI
    start_date = '2023-05-03T00:00:00',
    end_date = '2024-05-03T04:00:00',
    top=1000,
    attributes={'swathIdentifier': 'S1'} # Swath identifier Stripmap: S1, ..., S6 | 'WV1,WV2' for wavemode
    # base_url defaults to 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products'
    # order_by defaults to 'ContentDate/Start desc'
)

df = searcher.execute_query()
print(f"Number of results: {len(df)}")
# Display the first few rows of the DataFrame
searcher.display_results(top_n=15)

### CCM

In [ ]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

searcher = CopernicusDataSearcher()
# Configure the search parameters
searcher._query_by_filter(
    collection_name='CCM',
    product_type='NAO_MS4_2A_07B6',
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=None,  # Use the defined AOI
    start_date='2014-05-03T00:00:00',
    end_date='2024-05-03T04:00:00',
    top=1000,
    attributes=None
)

df = searcher.execute_query()
print(f"Number of results: {len(df)}")
# Display the first few rows of the DataFrame
searcher.display_results(top_n=15)

### Search by Product Name (Exact Match)

You can search for a specific product if you know its exact name.

**Tip:**

To search by exact product name, make sure the name exists in the Copernicus Data Space catalogue. You can find valid product names by running a general search first, or by browsing the [Copernicus Data Space Catalogue](https://dataspace.copernicus.eu/browser/). If your search returns no results, try using a product name from the official documentation or a recent search result.

Below, we use a known-valid product name as an example.

In [ ]:
from phidown.search import CopernicusDataSearcher

searcher_by_name = CopernicusDataSearcher()

# Replace with an actual product name you want to find
product_to_find = 'S1A_IW_GRDH_1SDV_20141031T161924_20141031T161949_003076_003856_634E.SAFE' # Example, replace with a recent, valid name

print(f"Searching for product with exact name: {product_to_find}\n")
df_exact = searcher_by_name.query_by_name(product_name=product_to_find)

if not df_exact.empty:
    searcher_by_name.display_results(top_n=1)
    display(df_exact)
else:
    print(f"Product '{product_to_find}' not found or an error occurred.")

## 📥 Download Your Product

Important:
If your credentials are not deleted automatically, you must manually revoke them here:

🔗 Manage S3 Credentials: https://eodata-s3keysmanager.dataspace.copernicus.eu/panel/s3-credentials

Stay secure — always clean up your credentials!

In [ ]:
from phidown.search import CopernicusDataSearcher

searcher = CopernicusDataSearcher() 

filename = 'S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE'
searcher.download_product(filename)

Alternative CLI:

In your command line 
```bash 
python -m phidown.downloader -u {username} -p {password} -eo_product_name {filename}
```


In [ ]:
from phidown.downloader import load_credentials
# Download the product using the command line
filename = 'S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE'
username, password = load_credentials()
# Run the command line downloader
# Note: You need to have the phidown package installed and available in your PATH
!python -m phidown.downloader -u {username} -p {password} -eo_product_name {filename}